# Introduction to CVXPY for Optimization

CVXPY is an open source Python-embedded modeling language for convex optimization problems. It lets you express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

It basically the "JuMP" in Python. If you really hate using Julia, then CVXPY is the alternative in Python that can help you out. 

The documentation of CVXPY https://www.cvxpy.org/

In [1]:
# Step 1: Install necessary packages
# !pip install cvxpy
# !pip install numpy
# !pip install gurobipy
# !pip install ortools==9.7.2996 (The version after 9.8 is not supported for cvxpy)

In [2]:
# Step 2: import cvxpy and numpy packages
import cvxpy as cp
import numpy as np

In [3]:
# Step 3: check installed solvers
print(cp.installed_solvers())

['CLARABEL', 'ECOS', 'ECOS_BB', 'GLOP', 'GUROBI', 'OSQP', 'PDLP', 'SCIPY', 'SCS']


## Section 1: Linear Optimization Problem

Define the optimization variables

In [4]:
x = cp.Variable()
y = cp.Variable()

Define the objective function and constraints

In [5]:
objective = cp.Maximize(3*x + 2*y)
constraints = [2*x + y <= 10, x + 3*y <= 12, x >= 0, y >= 0]

Create the optimization problem

In [6]:
linear_problem = cp.Problem(objective, constraints)

Solve the problem (We use the solver Gurobi but you can also use other solvers. )

In [7]:
linear_problem.solve(solver=cp.GUROBI,verbose=True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Jan 20 05:05:15 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Jan 20 05:05:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 20 05:05:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 20 05:05:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 20 05:05:15 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 20 05:05:15 PM: Compiling problem (target solver=GUROBI).
(CV

16.4

Display the results

In [8]:
print("Optimal value:", linear_problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)

Optimal value: 16.4
Optimal x: 3.5999999999999996
Optimal y: 2.8000000000000003


## Section 2: A First-Order LP Solver (PDLP)

In this section we solve a randomly generated "big" LP instance using cvxpy and different solvers.

The problem is 
$$
\min c^\top x, \ s.t. Ax \le b \ ,
$$
in which $A$ has $m$ rows and $n$ columns. 

The data is created as follows:

In [9]:
np.random.seed(123)
m = 5000
n = 3000

# Generate A, b, c so that (x0,y0) is an optimal primal-dual solution to the LP problem 
s0 = np.random.randn(m)
y0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ y0

# Define the LP using CVXPY
x = cp.Variable(n)
big_LP = cp.Problem(cp.Minimize(c.T@x), [A @ x <= b])

Solve the problem using the primal simplex method of GUROBI (about 700 seconds).

In [11]:
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=0)
solution=x.value
print(opt_val)
print(solution)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Jan 20 05:06:27 PM: Your problem has 3000 variables, 1 constraints, and 0 parameters.
(CVXPY) Jan 20 05:06:27 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 20 05:06:27 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 20 05:06:27 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 20 05:06:27 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 20 05:06:27 PM: Compiling problem (target solver=GUROBI).


Solve the problem using the dual simplex method of GUROBI (about 800 seconds.)

In [13]:
# Solve the problem using the default GUROBI dual simplex method
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=1)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Jan 20 05:18:12 PM: Your problem has 3000 variables, 1 constraints, and 0 parameters.
(CVXPY) Jan 20 05:18:12 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 20 05:18:12 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 20 05:18:12 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 20 05:18:12 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 20 05:18:12 PM: Using cached ASA map, for faster compilati

Solve the problem using PDLP (a first-order method for LP).  

The PDLP is the primal-dual hybrid gradient (PDHG) method with some practical enhancements. 
See more details of the mathematical background in https://developers.google.com/optimization/lp/pdlp_math. 

In [14]:
opt_val=big_LP.solve(solver=cp.PDLP, verbose=True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Jan 20 06:06:28 PM: Your problem has 3000 variables, 1 constraints, and 0 parameters.
(CVXPY) Jan 20 06:06:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 20 06:06:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 20 06:06:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 20 06:06:28 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 20 06:06:28 PM: Compiling problem (target solver=PDLP).
(C

Problem stats before presolving and rescaling:
There are 3000 variables, 5000 constraints, and 15000000 constraint matrix nonzeros.
Absolute values of nonzero constraint matrix elements: largest=5.381140, smallest=0.000000, avg=0.798038
Constraint matrix, infinity norm: max(row & col)=5.381140, min_col=3.082680, min_row=2.955656
Constraint bounds statistics (max absolute value per row): largest=220.898111, smallest=0.012123, avg=43.719825, l2_norm=3867.939661
Absolute values of objective vector elements: largest=200.838012, smallest=0.034710, avg=40.066409, l2_norm=2732.719799
Gaps between variable upper and lower bounds: #finite=0 of 3000, largest=0.000000, smallest=0.000000, avg=nan
Problem stats after presolving and rescaling:
There are 3000 variables, 5000 constraints, and 15000000 constraint matrix nonzeros.
Absolute values of nonzero constraint matrix elements: largest=0.071478, smallest=0.000000, avg=0.012799
Constraint matrix, infinity norm: max(row & col)=0.071478, min_col=0.0

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jan 20 06:08:01 PM: Problem status: optimal
(CVXPY) Jan 20 06:08:01 PM: Optimal value: 3.965e+03
(CVXPY) Jan 20 06:08:01 PM: Compilation took 1.982e+01 seconds
(CVXPY) Jan 20 06:08:01 PM: Solver (including time spent in interface) took 7.323e+01 seconds


  2112   2133.0   72.1 |  1.97064e-07  9.85880e-07  9.66494e-08 |  0.000762429   0.00269512  0.000766475 |      3964.73      3964.73 |      54.4745      48.8115
Termination reason: TERMINATION_REASON_OPTIMAL
Solution point type: POINT_TYPE_AVERAGE_ITERATE
Final solution stats:
 iter# kkt_pass   time | rel_prim_res rel_dual_res      rel_gap |   prim_resid   dual_resid      obj_gap |     prim_obj     dual_obj |  prim_var_l2  dual_var_l2
  2112   2133.0   72.1 |  1.97064e-07  9.85880e-07  9.66494e-08 |  0.000762429   0.00269512  0.000766475 |      3964.73      3964.73 |      54.4745      48.8115


It requires no more than 100 seconds!!!